In [1]:
from dataclasses import dataclass
import numpy as np
import torch as t
import torch.nn.functional as F
import sklearn.datasets

In [ ]:
NUM_SAMPLES = 100_000
NUM_FEATURES = 5
NOISE = 0.5
DEVICE = t.device("cuda" if t.cuda.is_available() else "cpu")
DEVICE

In [ ]:
all_X, all_y = sklearn.datasets.make_regression(n_samples=NUM_SAMPLES, n_features=NUM_FEATURES, noise=NOISE)
print(all_X.shape, all_y.shape)

In [ ]:
train_size = int(NUM_SAMPLES * 0.7)
val_size = int(NUM_SAMPLES * 0.2)
test_size = NUM_SAMPLES - train_size - val_size
print(train_size, val_size, test_size)

In [ ]:
train_X = all_X[:train_size]
train_y = all_y[:train_size]
trainset = t.utils.data.TensorDataset(t.from_numpy(train_X).to(t.float32), t.from_numpy(train_y).to(t.float32))

In [ ]:
val_X = all_X[train_size:train_size+val_size]
val_y = all_y[train_size:train_size+val_size]
valset = t.utils.data.TensorDataset(t.from_numpy(val_X).to(t.float32), t.from_numpy(val_y).to(t.float32))

In [ ]:
test_X = all_X[train_size+val_size:]
test_y = all_y[train_size+val_size:]
testset = t.utils.data.TensorDataset(t.from_numpy(test_X).to(t.float32), t.from_numpy(test_y).to(t.float32))

In [ ]:
class SimpleLR(t.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = t.nn.Linear(NUM_FEATURES, 3)
        self.fc2 = t.nn.Linear(3, 1)
        
    def forward(self, batch_x):
        x = F.relu(self.fc1(batch_x))
        batch_y_hat = self.fc2(x)
        return t.squeeze(batch_y_hat)

In [ ]:
@dataclass
class Hyperparams:
    batch_size: int = 10
    epochs: int = 10
    learning_rate: float = 0.0001

    def to_dict(self):
        return {
            "batch_size": self.batch_size,
            "epochs": self.epochs,
            "learning_rate": self.learning_rate
        }

In [ ]:
x, y = next(iter(trainset))
batch_of_one = t.unsqueeze(x, 0)
print(x.shape, batch_of_one.shape, x.dtype, y.shape)

In [ ]:
SimpleLR().forward(batch_of_one)

In [ ]:
dl = t.utils.data.DataLoader(valset, batch_size=5000)
X, y = next(iter(dl))
y_hat = SimpleLR().forward(X)
print(X.shape, y.shape, y_hat.shape)
rmse = t.sqrt(F.mse_loss(y, y_hat))
rmse

In [ ]:
def train(model, optim, loss_fn, epochs, trainloader, valloader):
    model = model.to(DEVICE)
    for epoch in range(epochs):
        # Process the training set
        train_losses = []
        train_outputs = t.tensor([], dtype=t.float32)
        train_targets = t.tensor([], dtype=t.float32)
        model.train()
        with t.enable_grad():
            for batch_X, batch_y in trainloader:
                batch_X = batch_X.to(DEVICE)
                batch_y = batch_y.to(DEVICE)

                optim.zero_grad()
                batch_y_hat = model.forward(batch_X)
                loss = loss_fn(batch_y_hat, batch_y)
                loss.backward()
                optim.step()

                train_losses.append(loss.detach())
                train_outputs = t.cat((train_outputs, batch_y_hat.detach()))
                train_targets = t.cat((train_targets, batch_y.detach()))
        train_loss = np.mean(train_losses)
        train_rmse = t.sqrt(F.mse_loss(train_outputs, train_targets))

        # Calculate the validation metrics
        val_losses = []
        val_outputs = t.tensor([], dtype=t.float32)
        val_targets = t.tensor([], dtype=t.float32)
        model.eval()
        with t.no_grad():
            for batch_X, batch_y in valloader:
                batch_X = batch_X.to(DEVICE)
                batch_y = batch_y.to(DEVICE)
                batch_y_hat = model(batch_X)
                loss = loss_fn(batch_y_hat, batch_y)
                val_losses.append(loss.detach())
                val_outputs = t.cat((val_outputs, batch_y_hat.detach()))
                val_targets = t.cat((val_targets, batch_y.detach()))
        val_loss = np.mean(val_losses)
        val_rmse = t.sqrt(F.mse_loss(val_outputs, val_targets))

        print(f"\nEpoch {epoch}:")
        print(f"Loss: train={train_loss:.3f}, validation={val_loss:.3f}")
        print(f"RMSE: train={train_rmse:.3f}, validaiton={val_rmse:.3f}")

In [ ]:
hparams = Hyperparams(batch_size=32, epochs=5, learning_rate=0.005)
model = SimpleLR()
optim = t.optim.Adam(model.parameters(), lr=hparams.learning_rate)
loss_fn = t.nn.MSELoss(reduction="mean")
trainloader = t.utils.data.DataLoader(trainset, batch_size=hparams.batch_size, shuffle=True)
valloader = t.utils.data.DataLoader(valset, batch_size=5000)

In [ ]:
train(model, optim, loss_fn, hparams.epochs, trainloader, valloader)

In [ ]:
testloader = t.utils.data.DataLoader(testset, batch_size=len(testset))
X, y = next(iter(testloader))
model.eval()
with t.no_grad():
    y_hat = model(X)
    test_rmse = t.sqrt(F.mse_loss(y_hat, y))
print(f"Test RMSE={test_rmse:.3f}")
print(list(zip(y[:5].numpy(), y_hat[:5].numpy())))